In [1]:
import pandas as pd

In [2]:
import copy

In [3]:
import dgutils.pandas as dgp

/Users/alicegao/anaconda2/envs/plot_py3/lib/python3.7/site-packages/genome_kit/data_manager.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
df = pd.read_pickle('../2020_09_22/data/rand_s1_bb_0p1.pkl.gz')

In [5]:
row = df.iloc[0]

In [6]:
row

ensemble_diversity                                                  9.7
free_energy                                                        -8.5
len                                                                  31
mfe_frequency                                                  0.459552
one_idx               ([0, 1, 2, 3, 4, 5, 9, 10, 11, 12, 19, 20, 21,...
seq                                     CUCGACCCCCUUGACGUCGCAAGUGUUGGGG
bounding_boxes        [((0, 24), (6, 6), stem), ((9, 19), (4, 4), st...
bb_stem               [{'bb_x': 0, 'bb_y': 29, 'siz_x': 6, 'siz_y': ...
bb_iloop              [{'bb_x': 5, 'bb_y': 22, 'siz_x': 5, 'siz_y': ...
bb_hloop              [{'bb_x': 5, 'bb_y': 15, 'siz_x': 11, 'siz_y':...
Name: 0, dtype: object

In [7]:
def process_bb_old_to_new(old_bb):
    # old_bb: list of:
    # (top_left_x, top_left_y), (siz_x, siz_y), bb_type
    df = []
    for (top_left_x, top_left_y), (siz_x, siz_y), bb_type in old_bb:
        bb_x = top_left_x
        bb_y = top_left_y + siz_y - 1
        df.append({
            'bb_x': bb_x,
            'bb_y': bb_y,
            'siz_x': siz_x,
            'siz_y': siz_y,
            'bb_type': bb_type
        })
    return pd.DataFrame(df)

In [8]:
df_target = process_bb_old_to_new(row['bounding_boxes'])

In [9]:
df_target

,bb_x,bb_y,siz_x,siz_y,bb_type
0,0,29,6,6,stem
1,9,22,4,4,stem
2,5,24,5,3,internal_loop
3,12,19,8,8,hairpin_loop


In [10]:
def add_bb_bottom_left(df):
    # add bottom left reference point
    # df: requires columns bb_x, bb_y, siz_x, siz_y
    
    def add_bl(bb_x, bb_y, siz_x, siz_y):
        x = bb_x + siz_x - 1
        y = bb_y - siz_y + 1
        return x, y
    
    df = dgp.add_columns(df, ['bl_x', 'bl_y'], ['bb_x', 'bb_y', 'siz_x', 'siz_y'], add_bl)
    return df

In [11]:
df_stem = pd.DataFrame(row['bb_stem'])
df_stem = add_bb_bottom_left(df_stem)

df_iloop = pd.DataFrame(row['bb_iloop'])
df_iloop = add_bb_bottom_left(df_iloop)

df_hloop = pd.DataFrame(row['bb_hloop'])
df_hloop = add_bb_bottom_left(df_hloop)

In [98]:
df_stem

,bb_x,bb_y,siz_x,siz_y,prob,bl_x,bl_y
0,0,29,6,6,"[0.41025621353158326, 0.44290844275857927, 0.2...",5,24
1,5,30,4,4,[0.20544070627534108],8,27
2,6,30,4,4,"[0.4038486250672587, 0.27864132850155837, 0.27...",9,27
3,9,22,4,4,"[0.6770828362027589, 0.573858923373826, 0.8764...",12,19
4,12,26,4,4,[0.09296749466427809],15,23
5,13,23,2,2,"[0.4102197996540831, 0.32933917103708865, 0.41...",14,22


In [99]:
df_iloop

,bb_x,bb_y,siz_x,siz_y,prob,bl_x,bl_y
0,5,22,5,3,"[0.48002284303088494, 0.38280278817052693, 0.1...",9,20
1,5,23,5,3,"[0.5652983093527412, 0.4536248120951613, 0.477...",9,21
2,5,24,5,3,"[0.22653995386454878, 0.4762805845032449, 0.40...",9,22
3,8,24,4,5,[0.025440477807378965],11,20
4,8,25,2,6,[0.237831903897844],9,20
5,8,25,5,6,[0.26275034653646495],12,20
6,8,26,2,6,[0.18583527858956853],9,21
7,8,26,5,2,[0.1551192200380325],12,25
8,8,26,5,6,[0.18746263337043842],12,21
9,8,27,2,6,[0.21078391904750435],9,22


In [100]:
df_hloop

,bb_x,bb_y,siz_x,siz_y,prob,bl_x,bl_y
0,5,15,11,11,"[0.5219362231427945, 0.21914758759218422, 0.24...",15,5
1,12,19,8,8,"[0.5592991751604471, 0.6358038212924311, 0.709...",19,12
2,14,22,9,9,"[0.5634663251435189, 0.2603944252215587, 0.666...",22,14


In [15]:
def compatible_counts(df1, df2, col1, col2, out_name):
    # join df1 and df2 on col1/col2, count the number of compatible entries
    # this is equivalent to:
    # for each row of df1, find how many rows there are in df2 that's compatible
    
    if isinstance(col1, str):
        col1 = [col1]
    if isinstance(col2, str):
        col2 = [col2]
    
    assert out_name not in col1 + col2
    
    # first aggregate count on df2 using col2
    df2_ct = df2[col2].groupby(col2).size().reset_index(name=out_name)
    # hack col name, to avoid duplication
    df2_ct = df2_ct.rename(columns={a: b for a, b in zip(col2, col1)})
    # join to df1
    df = pd.merge(df1, df2_ct, left_on=col1, right_on=col1, how='outer')
    # replace missing entry with 0 (count)
    df = df.fillna(0)
    
    return df

In [16]:
# for each iloop, check:
# how many compatible outer stems (stem.bottom_left == iloop.top_right)
df_iloop_cleanup = compatible_counts(df_iloop, df_stem, col1=['bb_x', 'bb_y'], col2=['bl_x', 'bl_y'], out_name='num_compatible_stem_outer')
# how many compatible inner stems (stem.top_right == iloop.bottom_left)
df_iloop_cleanup = compatible_counts(df_iloop_cleanup, df_stem, col1=['bl_x', 'bl_y'], col2=['bb_x', 'bb_y'], out_name='num_compatible_stem_inner')
# drop those rows without compatible stems on both ends
df_iloop_cleanup = df_iloop_cleanup[(df_iloop_cleanup['num_compatible_stem_inner'] > 0) & (df_iloop_cleanup['num_compatible_stem_outer'] > 0)]

In [17]:
df_iloop_cleanup

,bb_x,bb_y,siz_x,siz_y,prob,bl_x,bl_y,num_compatible_stem_outer,num_compatible_stem_inner
4,5.0,24.0,5.0,3.0,"[0.22653995386454878, 0.4762805845032449, 0.40...",9.0,22.0,1.0,1.0
5,8.0,27.0,2.0,6.0,[0.21078391904750435],9.0,22.0,1.0,1.0
10,8.0,27.0,5.0,2.0,[0.05582155712225877],12.0,26.0,1.0,1.0
17,9.0,27.0,5.0,5.0,"[0.10652713805518142, 0.0927118477820101]",13.0,23.0,1.0,1.0


In [18]:
# for each hloop, check:
# how many compatible outer stems (stem.bottom_left == iloop.top_right)
df_hloop_cleanup = compatible_counts(df_hloop, df_stem, col1=['bb_x', 'bb_y'], col2=['bl_x', 'bl_y'], out_name='num_compatible_stem_outer')
# drop those rows without compatible stem
df_hloop_cleanup = df_hloop_cleanup[df_hloop_cleanup['num_compatible_stem_outer'] > 0]
# drop those not symmetric & across diagonal
df_hloop_cleanup = df_hloop_cleanup[(df_hloop_cleanup['bb_x'] == df_hloop_cleanup['bl_y']) & (df_hloop_cleanup['bb_y'] == df_hloop_cleanup['bl_x']) & (df_hloop_cleanup['siz_x'] == df_hloop_cleanup['siz_y'])]

In [19]:
df_hloop_cleanup

,bb_x,bb_y,siz_x,siz_y,prob,bl_x,bl_y,num_compatible_stem_outer
1,12,19,8.0,8.0,"[0.5592991751604471, 0.6358038212924311, 0.709...",19.0,12.0,1.0
2,14,22,9.0,9.0,"[0.5634663251435189, 0.2603944252215587, 0.666...",22.0,14.0,1.0


In [20]:
df_stem

,bb_x,bb_y,siz_x,siz_y,prob,bl_x,bl_y
0,0,29,6,6,"[0.41025621353158326, 0.44290844275857927, 0.2...",5,24
1,5,30,4,4,[0.20544070627534108],8,27
2,6,30,4,4,"[0.4038486250672587, 0.27864132850155837, 0.27...",9,27
3,9,22,4,4,"[0.6770828362027589, 0.573858923373826, 0.8764...",12,19
4,12,26,4,4,[0.09296749466427809],15,23
5,13,23,2,2,"[0.4102197996540831, 0.32933917103708865, 0.41...",14,22


In [21]:
class LocalStructureBb(object):
    
    def __init__(self, top_right_x, top_right_y, size_x, size_y, bb_id, bb_type):
        self.id = bb_id
        assert bb_type in ['stem', 'iloop', 'hloop']
        self.type = bb_type
        self.tr_x = top_right_x
        self.tr_y = top_right_y
        self.size_x = size_x
        self.size_y = size_y
        # also store bottom left
        self.bl_x = top_right_x + size_x - 1
        self.bl_y = top_right_y - size_y + 1

    def share_top_right_corner(self, another_bb):
        # check if self.top_right == another_bb.bottom_left
        if self.tr_x == another_bb.bl_x and self.tr_y == another_bb.bl_y:
            return True
        else:
            return False
        
    def share_bottom_left_corner(self, another_bb):
        # check if self.bottom_left == another_bb.top_right
        if self.bl_x == another_bb.tr_x and self.bl_y == another_bb.tr_y:
            return True
        else:
            return False
        
    def overlaps(self, another_bb):
        raise NotImplementedError
        
    def bp_conflict(self, another_bb):
        # only makes sense for stem bb comparison
        # where the base pair ranges are in conflict with each other
        # i.e. if another_bb cannot be included if self is included in global structure
        # x range
        x1_1, x1_2 = self.tr_x, self.bl_x
        x2_1, x2_2 = another_bb.tr_x, another_bb.bl_x
        x_range_conflict = (x1_1 <= x2_1 <= x1_2) or (x2_1 <= x1_1 <= x2_2)
        # y range
        y1_1, y1_2 = self.bl_y, self.tr_y
        y2_1, y2_2 = another_bb.bl_y, another_bb.tr_y
        y_range_conflict = (y1_1 <= y2_1 <= y1_2) or (y2_1 <= y1_1 <= y2_2)
        return x_range_conflict or y_range_conflict
    
    def __repr__(self):
        return f"{self.type} {self.id} top right ({self.tr_x}, {self.tr_y}), bottom left ({self.bl_x}, {self.bl_y})"

In [22]:
# bb objects
# use enumerate on df since we want to contiguous ids, not the original df index
stems = [LocalStructureBb(row['bb_x'], row['bb_y'], row['siz_x'], row['siz_y'], f'stem_{idx}', 'stem') for idx, (_, row) in enumerate(df_stem.iterrows())]
iloops = [LocalStructureBb(row['bb_x'], row['bb_y'], row['siz_x'], row['siz_y'], f'iloop_{idx}', 'iloop') for idx, (_, row) in enumerate(df_iloop_cleanup.iterrows())]
hloops = [LocalStructureBb(row['bb_x'], row['bb_y'], row['siz_x'], row['siz_y'], f'hloop_{idx}', 'hloop') for idx, (_, row) in enumerate(df_hloop_cleanup.iterrows())]


In [23]:
stems

[stem stem_0 top right (0, 29), bottom left (5, 24),
 stem stem_1 top right (5, 30), bottom left (8, 27),
 stem stem_2 top right (6, 30), bottom left (9, 27),
 stem stem_3 top right (9, 22), bottom left (12, 19),
 stem stem_4 top right (12, 26), bottom left (15, 23),
 stem stem_5 top right (13, 23), bottom left (14, 22)]

In [24]:
stems[0].bp_conflict(stems[4])

True

In [25]:
iloops

[iloop iloop_0 top right (5.0, 24.0), bottom left (9.0, 22.0),
 iloop iloop_1 top right (8.0, 27.0), bottom left (9.0, 22.0),
 iloop iloop_2 top right (8.0, 27.0), bottom left (12.0, 26.0),
 iloop iloop_3 top right (9.0, 27.0), bottom left (13.0, 23.0)]

In [26]:
hloops

[hloop hloop_0 top right (12, 19), bottom left (19.0, 12.0),
 hloop hloop_1 top right (14, 22), bottom left (22.0, 14.0)]

In [27]:
class OneStepChain(object):
    
    def __init__(self, bb, next_bb=None):
        self.bb = bb
        self.next_bb = next_bb
        
    def clear_next_bb(self):
        self.next_bb = None
    
    def add_next_bb(self, next_bb, validate=True):
        if validate:
            assert self.bb.share_top_right_corner(next_bb)
        if self.next_bb is None:
            self.next_bb = []
        self.next_bb.append(next_bb)
        
    def __repr__(self):
        if self.next_bb:
            tmp = [x.id for x in self.next_bb]
        else:
            tmp = 'N/A'
        return f"{self.bb}. Next: {tmp}"

In [28]:
class GlobalConstraint(object):
    
    def __init__(self, stems, iloops, hloops):
        raise NotImplementedError
    
    def conflict(self, bb1, bb2):
        # check bb1 and bb2 are in the known bb's
        # same type?
        # check overlap and bp conflict
        raise NotImplementedError

In [29]:
# find next compatible, start with iloop
iloop_os_chain = []
for iloop in iloops:
    iloop_os = OneStepChain(iloop)
    for stem in stems:
        if iloop.share_top_right_corner(stem):
            iloop_os.add_next_bb(stem)
    iloop_os_chain.append(iloop_os)

In [30]:
# find next compatible, start with stem
stem_os_chain = []
for stem in stems:
    stem_os = OneStepChain(stem)
    for iloop in iloops:
        if stem.share_top_right_corner(iloop):
            stem_os.add_next_bb(iloop)
    stem_os_chain.append(stem_os)

In [31]:
# find next compatible, start with hloop
hloop_os_chain = []
for hloop in hloops:
    hloop_os = OneStepChain(hloop)
    for stem in stems:
        if hloop.share_top_right_corner(stem):
            hloop_os.add_next_bb(stem)
    hloop_os_chain.append(hloop_os)

In [32]:
iloop_os_chain

[iloop iloop_0 top right (5.0, 24.0), bottom left (9.0, 22.0). Next: ['stem_0'],
 iloop iloop_1 top right (8.0, 27.0), bottom left (9.0, 22.0). Next: ['stem_1'],
 iloop iloop_2 top right (8.0, 27.0), bottom left (12.0, 26.0). Next: ['stem_1'],
 iloop iloop_3 top right (9.0, 27.0), bottom left (13.0, 23.0). Next: ['stem_2']]

In [33]:
stem_os_chain

[stem stem_0 top right (0, 29), bottom left (5, 24). Next: N/A,
 stem stem_1 top right (5, 30), bottom left (8, 27). Next: N/A,
 stem stem_2 top right (6, 30), bottom left (9, 27). Next: N/A,
 stem stem_3 top right (9, 22), bottom left (12, 19). Next: ['iloop_0', 'iloop_1'],
 stem stem_4 top right (12, 26), bottom left (15, 23). Next: ['iloop_2'],
 stem stem_5 top right (13, 23), bottom left (14, 22). Next: ['iloop_3']]

In [34]:
hloop_os_chain

[hloop hloop_0 top right (12, 19), bottom left (19.0, 12.0). Next: ['stem_3'],
 hloop hloop_1 top right (14, 22), bottom left (22.0, 14.0). Next: ['stem_5']]

In [35]:
# for convenience
os_chain = {x.bb.id: x for x in iloop_os_chain + stem_os_chain + hloop_os_chain}

In [36]:
os_chain

{'iloop_0': iloop iloop_0 top right (5.0, 24.0), bottom left (9.0, 22.0). Next: ['stem_0'],
 'iloop_1': iloop iloop_1 top right (8.0, 27.0), bottom left (9.0, 22.0). Next: ['stem_1'],
 'iloop_2': iloop iloop_2 top right (8.0, 27.0), bottom left (12.0, 26.0). Next: ['stem_1'],
 'iloop_3': iloop iloop_3 top right (9.0, 27.0), bottom left (13.0, 23.0). Next: ['stem_2'],
 'stem_0': stem stem_0 top right (0, 29), bottom left (5, 24). Next: N/A,
 'stem_1': stem stem_1 top right (5, 30), bottom left (8, 27). Next: N/A,
 'stem_2': stem stem_2 top right (6, 30), bottom left (9, 27). Next: N/A,
 'stem_3': stem stem_3 top right (9, 22), bottom left (12, 19). Next: ['iloop_0', 'iloop_1'],
 'stem_4': stem stem_4 top right (12, 26), bottom left (15, 23). Next: ['iloop_2'],
 'stem_5': stem stem_5 top right (13, 23), bottom left (14, 22). Next: ['iloop_3'],
 'hloop_0': hloop hloop_0 top right (12, 19), bottom left (19.0, 12.0). Next: ['stem_3'],
 'hloop_1': hloop hloop_1 top right (14, 22), bottom lef

In [104]:
class FullChain(object):
    
    def __init__(self, start):
        # make sure start is either stem or hloop
        assert start.type in ['stem', 'hloop']
        self.chain = (start, )
        self.start = start
        self.completed = False
    
    def add_bb(self, bb):
        last_bb = self.chain[-1]
        assert last_bb.share_top_right_corner(bb), f"{last_bb} {bb}"
        self.chain = self.chain + (bb,)
    
    def complete(self, validate=True):
        if validate:
            end = self.chain[-1]
            assert end.type == 'stem'
        self.end = end
        self.completed = True
    
    def merge_chain(self, another_chain):
        # merge another (completed) chain
        assert another_chain.completed
        # make sure they share the same bb
        assert self.chain[-1] == another_chain.chain[0]
        self.chain = self.chain + another_chain.chain[1:]
        self.end = another_chain.end
        self.completed == True
        
    def __repr__(self):
        status = "Completed" if self.completed else "Incomplete"
        return f"FullChain {self.chain} {status}"
#         return f"FullChain {[x.id for x in self.chain]} {status}"

In [105]:
# for convenience, keep track of chains starting with each bb
# not used for now, will need this to improve efficiency
bb2chain = {bb: [] for bb in stems + iloops + hloops} 
# all chains (some might be subset of others)
full_chains = []


def grow_chain(chain):        
    # look up last item in chain
    this_bb = chain.chain[-1]
    # check its next compatible elements
    next_bbs = os_chain[this_bb.id].next_bb
    # finish if empty
    # otherwise add next item & recursion
    if next_bbs is None:
        tmp = copy.copy(chain)
        tmp.complete()
        full_chains.append(tmp)
        return
    else:
#         print(next_bbs, '\n')
        # if end with stem, make a copy, add to full chain list
        if chain.chain[-1].type == 'stem':
            tmp = copy.copy(chain)
            tmp.complete()
            full_chains.append(tmp)
        
#         for i in range(len(os_chain[chain.chain[-1].id].next_bb)):
        for i in range(len(next_bbs)):
            # operate on copy before 'branching out' to avoid messing up with a global 'chain'
            tmp = copy.copy(chain)
            tmp.add_bb(next_bbs[i])
            grow_chain(tmp)


# start with stem or hloop
for x in stems + hloops:
#     print(x)
    fc = FullChain(x)
    grow_chain(fc)




In [106]:
full_chains

[FullChain (stem stem_0 top right (0, 29), bottom left (5, 24),) Completed,
 FullChain (stem stem_1 top right (5, 30), bottom left (8, 27),) Completed,
 FullChain (stem stem_2 top right (6, 30), bottom left (9, 27),) Completed,
 FullChain (stem stem_3 top right (9, 22), bottom left (12, 19),) Completed,
 FullChain (stem stem_3 top right (9, 22), bottom left (12, 19), iloop iloop_0 top right (5.0, 24.0), bottom left (9.0, 22.0), stem stem_0 top right (0, 29), bottom left (5, 24)) Completed,
 FullChain (stem stem_3 top right (9, 22), bottom left (12, 19), iloop iloop_1 top right (8.0, 27.0), bottom left (9.0, 22.0), stem stem_1 top right (5, 30), bottom left (8, 27)) Completed,
 FullChain (stem stem_4 top right (12, 26), bottom left (15, 23),) Completed,
 FullChain (stem stem_4 top right (12, 26), bottom left (15, 23), iloop iloop_2 top right (8.0, 27.0), bottom left (12.0, 26.0), stem stem_1 top right (5, 30), bottom left (8, 27)) Completed,
 FullChain (stem stem_5 top right (13, 23), b

In [96]:
len(full_chains)

15

In [115]:
import numpy as np

In [108]:
# from scipy.spatial.distance import pdist

In [120]:
# all pairwise compatibility of stems
# dm = pdist(pd.DataFrame({'stem': stems}).values, lambda x, y: x.bp_conflict(y))

distances = np.zeros((len(stems), len(stems)), dtype=object)
for i in range(len(stems)):
    for j in range(len(stems)):
        d = stems[i].bp_conflict(stems[j])
        distances[i, j] = d
        distances[j, i] = d

        
stem_ids = [x.id for x in stems]
df_stem_conflict = pd.DataFrame(distances, index=stem_ids, columns=stem_ids)
        

In [121]:
df_stem_conflict

,stem_0,stem_1,stem_2,stem_3,stem_4,stem_5
stem_0,True,True,True,False,True,False
stem_1,True,True,True,False,False,False
stem_2,True,True,True,True,False,False
stem_3,False,False,True,True,True,True
stem_4,True,False,False,True,True,True
stem_5,False,False,False,True,True,True


In [ ]:
# each 'chain' can start with stem or hloop (although there will be further implication if starting with stem),
# and can only end with stem
# to combine multiple chains in the same global structure, global constraints need to be satisfied (no overlapping OS, no bp conflict between stems)



In [ ]:
# DP on growing the 'chain'
# e.g. if we already know the tree 'growing' from stem_a we don't need to recompute it

In [ ]:
# incompatible stems (local constraints)



In [ ]:
# 'implied' loops:
# once we put together a 'global structure', any implicit loop (from choice of stems)
# should also be included
# this can be done by naively:
# apply stems -> binary array -> run script to find local bb -> check it's the same

In [ ]:
# assembly of 'stretches':
# can start with stem or hloop (starting with stem implies it's connected to non-local structure <- to be verified later?)

In [ ]:
# clean up

# add bottom left corner (1-index) for easy comparison

# any bb out of range



# rank by: number of proposals, probabilities